## Test environment

In [1]:
# Import modules
from multiplexdesigner.designer.design import design_primers
from multiplexdesigner.designer.multiplexpanel import panel_factory

In [2]:
# Get the mutation list (junctions) around which to design primer pairs
fasta_file = "/Users/ctosimsen/Documents/hg38/hg38.fa"
config_file = "./config/designer_default_config.json"
design_input_file = "./data/junctions.csv"

The `panel_factory` creates a panel object with suitable design regions obtained from the reference genome, as well a logger, which is passed through to write information to the log file. Both get then passed to the design engine, which will design candidate primers based on the chosen algorithm `simsen` or `primer3`.

In [3]:
panel = design_primers(
    multiplexpanel=panel_factory(
        name="test_panel",
        genome="hg38",
        design_input_file=design_input_file,
        fasta_file=fasta_file,
        config_file=config_file,
        padding=200,
    ),
    method="simsen",
)

Successfully imported 20 junctions from ./data/junctions.csv
Merging junctions within 100 bp on same chromosome...
Merged 2 junctions: CLTCL1_p.R354H_CLTCL1_p.R354H
After merging: 19 junctions remain
Extracting design regions from /Users/ctosimsen/Documents/hg38/hg38.fa with 200bp padding...
Successfully extracted 19 design regions
Calculated junction coordinates for 19 junctions


## Exploring the panel object

Each panel contains one or more junctions/targets. Primers are designed left and right of each target first and then suitable primers pairs for each target are selcted from the designs.

Individual primer picking evaluates basic properties (length, Gc-content, Tm, fraction bound) as well thermodynamic properties (self and 3' complementarity). Primers receive a penalty based on the configuration file provided. suitable pairs are then selected by finding pairs of primers that can form a permissible amplicon. For ctDNA applications, where short amplicons are preferable, most pairs will be removed due to too long amplicons.

In [4]:
panel.junctions[0].primer_pairs[2].forward

Primer(name='WLS_p.I360N _1317_forward', seq='AGACATTCTCACCTTGACATCTCAG', direction='forward', start=154, length=25, bound=53.7, tm=60.3, tm_primer3=None, gc=44.0, penalty=44.6, self_any_th=-76.24, self_end_th=-89.44, hairpin_th=0.0, end_stability=3.35, engine='custom')

In [5]:
panel.junctions[0].primer_pairs[2].reverse

Primer(name='WLS_p.I360N _99_reverse', seq='TGCGCTGCCATGACTGTC', direction='reverse', start=211, length=18, bound=56.0, tm=60.7, tm_primer3=None, gc=61.11, penalty=43.7, self_any_th=9.73, self_end_th=-0.41, hairpin_th=0.0, end_stability=3.51, engine='custom')

In [6]:
panel.junctions[0].primer_pairs[0]

PrimerPair(forward=Primer(name='WLS_p.I360N _1317_forward', seq='AGACATTCTCACCTTGACATCTCAG', direction='forward', start=154, length=25, bound=53.7, tm=60.3, tm_primer3=None, gc=44.0, penalty=44.6, self_any_th=-76.24, self_end_th=-89.44, hairpin_th=0.0, end_stability=3.35, engine='custom'), reverse=Primer(name='WLS_p.I360N _100_reverse', seq='GCGCTGCCATGACTGTCA', direction='reverse', start=210, length=18, bound=56.0, tm=60.7, tm_primer3=None, gc=61.11, penalty=43.7, self_any_th=13.6, self_end_th=13.6, hairpin_th=0.0, end_stability=3.58, engine='custom'), insert_size=31, amplicon_sequence='AGACATTCTCACCTTGACATCTCAGGGTCCACTTACCTGACTAACGATGAAGAAGATGACAGTCATGGCAGCGC', amplicon_length=74, pair_penalty=116.30000000000001)

In [7]:
panel.junctions[0].primer_pairs[0].amplicon_sequence

'AGACATTCTCACCTTGACATCTCAGGGTCCACTTACCTGACTAACGATGAAGAAGATGACAGTCATGGCAGCGC'

In [8]:
(
    len(panel.junctions[0].primer_pairs[1].amplicon_sequence)
    - len("AGACATTCTCACCTTGACATCTCAG")
    - len("TGCGCTGCCATGACTGTCA")
)

31

In [9]:
for junction in panel.junctions:
    if hasattr(junction, "design_region"):
        print(f"\n{junction.name}:")
        print(f"  Region: {junction.design_region} ")


WLS_p.I360N :
  Region: GGTTTTACGGAAAGAGAAAATTTAGTCCATTTGGCCAGGTGATATCAAGACCCAGAAAAATATGGCTTGACCAGCTGTCCAGAATTATGGCCTCTCTCCTCCTTTCCTCAGTGGCTCTATTGAATTTCTGTGCAGGGTAGAGGGATCTCTGCAGAGACATTCTCACCTTGACATCTCAGGGTCCACTTACCTGACTAACGATGAAGAAGATGACAGTCATGGCAGCGCAGGCCAAGGTGATAAGCATGAGGAACTTGAACCTAAAAATTAGCCCCTATTAGAAAAGAAAGAGTAGTTTAATACTCCATCAGCTACCAATCCTTTTCTCACTATGTAAATCTATAAAAAGCTTAATTTTAAAGAAATCTGTAAAACCAAATCCCTAAGAATGACAGTAAAAT 

NOLC1_p.I687V:
  Region: CCTTGAGCAGGGAGTAGAAAGAATAAAGTGACAGGGCTCCAGCATGGTCCTCCTCTGTGTTAATCTCCCTCTCTACTTACCAGCGAGGTGCAGCCGGAGACTGGGGAGAGCGAGCCAATCAGGTTTTGAAGTTCACCAAAGGCAAGTCCTTTCGGCATGAGAAAACCAAGAAGAAGCGGGGCAGCTACCGGGGAGGCTCAATCTCTGTCCAGGTCAATTCTATTAAGTTTGACAGCGAGTGACCTGAGGCCATCTTCGGTGAAGCAAGGGTGATGATCGGAGACTACTTACTTTCTCCAGTGGACCTGGGAACCCTCAGGTCTCTAGGTGAGGGTCTTGATGAGGACAGAAGTTTAGAGTAGGTCCTAAGACTTTACAGTGTAACATCCTCTCTGGTCCTT 

DCDC1_p.T1379fs*2:
  Region: AGCAGCATTTAAGTGATAGCCACTTACCATGGGGAATGTTCCAGCCACAATTAACTTCCCATTACGATACCCATCCCCATTTTTGTATGCAATTATTTTCACTGCCTTCTGGT

In [15]:
import primer3

tm_result = primer3.bindings.calcHeterodimer(
    "AGACATTCTCACCTTGACATCTCAG", "TGCGCTGCCATGACTGTC", output_structure=True
)

In [16]:
tm_result.tm

-9.069075547323507

In [17]:
tm_result.structure_found

True

In [18]:
tm_result.dg

-2946.731144614112

In [21]:
tm_result.ascii_structure_lines

['SEQ\tA    T  TCACCTTGACATCTCAG',
 'SEQ\t GACA TC',
 'STR\t CTGT AG',
 'STR\t     C  TACCGTCGCGT------']